In [1]:
import os
import sys
sys.path.append('./src')
sys.path.append('../../') # to get bindings
#sys.path.append('./dependencies/indy-ros2/indy_description/urdf_files')

from utils import *
from gato_mpc import *
from gato_mpc_batch import *
import pinocchio as pin
from pinocchio.visualize import MeshcatVisualizer
import meshcat.geometry as g
import meshcat.transformations as tf
import numpy as np
np.set_printoptions(linewidth=99999999)

# Load robot model
model_dir = "description/"
urdf_path = model_dir + "indy7.urdf"
model, visual_model, collision_model = load_robot_model(urdf_path, model_dir)
model.gravity = pin.Motion.Zero()

# Create MPC solver and optimizer
#controller = MPC_GATO(model, N=32, dt=0.01)
controller = MPC_GATO_Batch(model, N=64, dt=0.01, batch_size=1)


In [2]:
# initial state
#xstart = np.zeros(controller.nx)
xstart = np.hstack((np.ones(controller.nq), np.zeros(controller.nv)))

# goal ee positions
endpoints = np.array([
#   np.array(controller.eepos(np.ones(controller.nq))),
    np.array(controller.eepos(np.zeros(controller.nq))), 
#   np.array(controller.eepos(-0.9*np.ones(controller.nq)))
])

# Run MPC
q_trajectory, mpc_stats = controller.run_mpc(xstart, endpoints, 2.5)

Running MPC with 500 steps, dt: 0.005
Endpoints:
[[ 1.51468905e-10 -1.86500000e-01  1.26750000e+00]]
t:    0.000, goal dist:  1.07009
t:    0.050, goal dist:  1.08038
t:    0.100, goal dist:  1.08407
t:    0.200, goal dist:  0.98652
t:    0.350, goal dist:  0.67708
t:    0.400, goal dist:  0.55359
t:    0.550, goal dist:  0.23897
t:    0.700, goal dist:  0.13779
t:    0.800, goal dist:  0.15690
t:    0.950, goal dist:  0.17785
t:    1.100, goal dist:  0.18295
t:    1.150, goal dist:  0.18311
t:    1.350, goal dist:  0.18669
t:    1.400, goal dist:  0.18735
t:    1.600, goal dist:  0.18809
t:    1.650, goal dist:  0.18874
t:    1.900, goal dist:  0.19082
t:    2.200, goal dist:  0.18931
t:    2.300, goal dist:  0.18990
t:    2.450, goal dist:  0.18924
Maximum steps reached without convergence


In [1]:
for key in mpc_stats:
    print(key)
    print(mpc_stats[key])

NameError: name 'mpc_stats' is not defined

In [3]:
stats = controller.solver.get_stats()

for key in stats:
    print(key)
    print(stats[key])
    if key == 'solve_time':
        print(np.mean(stats[key]['values']))
        print(np.max(stats[key]['values']))
        print(np.min(stats[key]['values']))
    

solve_time
{'values': [47409.0, 38919.0, 51066.0, 161628.0, 42504.0, 50066.0, 143113.0, 136981.0, 138086.0, 143081.0, 124106.0, 113429.0, 128068.0, 141617.0, 137711.0, 89040.0, 97043.0, 84498.0, 100189.0, 69268.0, 55089.0, 82640.0, 65975.0, 57975.0, 52031.0, 33406.0, 42484.0, 70428.0, 37336.0, 38781.0, 34125.0, 52966.0, 43329.0, 44730.0, 31351.0, 45607.0, 59515.0, 58558.0, 23314.0, 68431.0, 53111.0, 43301.0, 32901.0, 62694.0, 58870.0, 56879.0, 41629.0, 48754.0, 63027.0, 48195.0, 50304.0, 62535.0, 40388.0, 50603.0, 22412.0, 64063.0, 68094.0, 61406.0, 66086.0, 61128.0, 21755.0, 56365.0, 59918.0, 59085.0, 36343.0, 86428.0, 63368.0, 60099.0, 69360.0, 61482.0, 71554.0, 69144.0, 67278.0, 36894.0, 41460.0, 74864.0, 61956.0, 43837.0, 57102.0, 89926.0, 40895.0, 97306.0, 61138.0, 76508.0, 61761.0, 80474.0, 83740.0, 39533.0, 38284.0, 89912.0, 48614.0, 31708.0, 67419.0, 28958.0, 68973.0, 87899.0, 115054.0, 112953.0, 72450.0, 39065.0, 35332.0, 48364.0, 36584.0, 35313.0, 37950.0, 23265.0, 25555.0, 5

In [4]:
# Setup pinocchio visualizer
viz = MeshcatVisualizer(model, collision_model, visual_model)
viz.initViewer()
viz.loadViewerModel()
# Visualize goals
for i, p in enumerate(endpoints):
    viz.viewer[f'point{i}'].set_object(
        g.Sphere(0.05), 
        g.MeshLambertMaterial(color=0xff0000))

    T = tf.translation_matrix(np.array(p[:]))
    viz.viewer[f'point{i}'].set_transform(T)
#print_stats(controller.solver.get_stats())
while 1:
    viz.play(q_trajectory=q_trajectory, dt=0.01)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7009/static/


KeyboardInterrupt: 

In [5]:
import pickle

print(q_trajectory)

# save trajectory to file
with open('q_trajectory.pkl', 'wb') as f:
    pickle.dump(q_trajectory, f)

[array([-0.10000215, -0.09995034, -0.10001428, -0.09973532, -0.10049627, -0.09970045]), array([-0.09999626, -0.0998874 , -0.10006218, -0.09946834, -0.1010091 , -0.09937014]), array([-0.09995853, -0.09966929, -0.10040356, -0.09892561, -0.10215607, -0.09848081]), array([-0.09993001, -0.09951316, -0.10071102, -0.09867053, -0.10275819, -0.09793127]), array([-0.10019403, -0.09893406, -0.10222653, -0.09829681, -0.10427466, -0.09615428]), array([-0.1005488 , -0.0984754 , -0.10358971, -0.09826648, -0.10516703, -0.09485713]), array([-0.10239259, -0.09691284, -0.10887264, -0.09938004, -0.10757959, -0.08959294]), array([-0.10401465, -0.09575326, -0.11306962, -0.10072823, -0.10908071, -0.08537751]), array([-0.10919641, -0.09250383, -0.12536881, -0.10528646, -0.11285667, -0.07426548]), array([-0.11285042, -0.09038437, -0.13363976, -0.10860579, -0.11509966, -0.06769667]), array([-0.12233224, -0.08490463, -0.1553781 , -0.11750124, -0.11957864, -0.05210192]), array([-0.12813764, -0.08150659, -0.168991